In [0]:
%sql
-- 1. Create a Master Reporting View (The Analytics Engine)
-- This combines all data and adds logic for "Length of Stay" (mocked) and "Procedure Intensity"
CREATE OR REPLACE TABLE healthcare_catalog.raw_data.gold_comprehensive_analytics AS
SELECT 
    p.patient_id,
    p.name,
    p.gender,
    e.encounter_id,
    e.admission_date,
    e.encounter_type,
    e.diagnosis,
    -- INDUSTRY METRIC: Procedure Count per visit
    COUNT(pr.procedure_id) as procedure_count,
    -- INDUSTRY METRIC: Resource Intensity Score (High if > 2 procedures)
    CASE WHEN COUNT(pr.procedure_id) > 2 THEN 'High Intensity' ELSE 'Routine' END as intensity_level,
    -- INDUSTRY METRIC: Length of Stay (Mocked for project: Inpatient stays are 3 days, Outpatient 1)
    CASE WHEN e.encounter_type = 'Inpatient' THEN 3 ELSE 1 END as length_of_stay_days
FROM healthcare_catalog.raw_data.silver_patients p
JOIN healthcare_catalog.raw_data.silver_encounters e ON p.patient_id = e.patient_id
LEFT JOIN healthcare_catalog.raw_data.silver_procedures pr ON e.encounter_id = pr.encounter_id
GROUP BY 1, 2, 3, 4, 5, 6, 7;

-- ---------------------------------------------------------
-- METRIC 1 & 2: EXECUTIVE SCORECARD (The "Big Boxes")
-- ---------------------------------------------------------
CREATE OR REPLACE TABLE healthcare_catalog.raw_data.gold_scorecard_metrics AS
SELECT 
    COUNT(DISTINCT patient_id) as total_unique_patients,
    COUNT(encounter_id) as total_encounters,
    ROUND(AVG(length_of_stay_days), 1) as avg_length_of_stay,
    SUM(procedure_count) as total_procedures_performed
FROM healthcare_catalog.raw_data.gold_comprehensive_analytics;

-- ---------------------------------------------------------
-- METRIC 3: PATIENT VOLUME BY TYPE (Pie/Donut Chart)
-- ---------------------------------------------------------
CREATE OR REPLACE TABLE healthcare_catalog.raw_data.gold_encounter_distribution AS
SELECT 
    encounter_type,
    COUNT(*) as visit_count,
    ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM healthcare_catalog.raw_data.gold_comprehensive_analytics), 1) as percentage
FROM healthcare_catalog.raw_data.gold_comprehensive_analytics
GROUP BY 1;

-- ---------------------------------------------------------
-- METRIC 4: DIAGNOSIS RANKING (Horizontal Bar Chart)
-- ---------------------------------------------------------
CREATE OR REPLACE TABLE healthcare_catalog.raw_data.gold_top_diagnoses AS
SELECT 
    diagnosis,
    COUNT(*) as total_cases,
    AVG(procedure_count) as avg_procedures
FROM healthcare_catalog.raw_data.gold_comprehensive_analytics
GROUP BY 1
ORDER BY total_cases DESC;

-- ---------------------------------------------------------
-- METRIC 5: ADMISSION TRENDS (Time Series/Line Chart)
-- ---------------------------------------------------------
CREATE OR REPLACE TABLE healthcare_catalog.raw_data.gold_daily_admissions AS
SELECT 
    admission_date,
    COUNT(*) as admission_count
FROM healthcare_catalog.raw_data.gold_comprehensive_analytics
GROUP BY 1
ORDER BY admission_date ASC;

-- ---------------------------------------------------------
-- METRIC 6: DEMOGRAPHIC HEATMAP (Gender vs Intensity)
-- ---------------------------------------------------------
CREATE OR REPLACE TABLE healthcare_catalog.raw_data.gold_demographic_intensity AS
SELECT 
    gender,
    intensity_level,
    COUNT(*) as visit_count
FROM healthcare_catalog.raw_data.gold_comprehensive_analytics
GROUP BY 1, 2;